# A simple notebook to read EOSC data

In [ ]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cmcrameri.cm as cmc
import pandas as pd

## Getting credentials for S3

In [ ]:
# Create the right credentials for the storage and read the data

auth_url = "https://identity.cloud.muni.cz/v3"

project_id = "XXXX"
identity_provider="egi.eu"
protocol="openid"

print(". ~/.bashrc")
print("mamba activate /cvmfs/notebooks.egi.eu/fedcloud/")

print('export OS_AUTH_URL="%s"' % auth_url)
print('export OS_PROTOCOL="%s"' % protocol)
print('export OS_PROJECT_ID="%s"' % project_id)
print('export OS_AUTH_TYPE="v3oidcaccesstoken"')
print('export OS_IDENTITY_PROVIDER="%s"' % identity_provider)
print('export OS_ACCESS_TOKEN=`cat /var/run/secrets/egi.eu/access_token`')

print('openstack ec2 credentials list')

In [ ]:
import xarray as xr
import s3fs


fs = s3fs.S3FileSystem(anon=True,
      client_kwargs={
         'endpoint_url': 'https://object-store.cloud.muni.cz'
      })


fs.ls('DT-demo')


In [ ]:
s3path = 's3://DT-demo/CAMS-PM2_5-20211222.netcdf'


ds = xr.open_dataset(fs.open(s3path))

ds

In [ ]:
ds.keys()

print(ds.longitude)

ds.coords['longitude'] = (ds['longitude'] + 180) % 360 - 180



In [ ]:
print(ds.longitude)

In [ ]:





import numpy as np

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt


fig = plt.figure(1, figsize=[15,10])

# We're using cartopy to project our data.
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.Mercator())
ax.coastlines(resolution='10m')

# We need to project our data to the new projection and for this we use `transform`.
# we set the original data projection in transform (here PlateCarree)
ds.sel(time=(np.timedelta64(2,'D') + np.timedelta64(12,'h')))['pm2p5_conc'].plot(ax=ax,
                                                                                 transform=ccrs.PlateCarree(),
                                                                                 vmin = 0, vmax = 35,
                                                                                 cmap=cmc.roma_r)
# One way to customize your title
plt.title("Copernicus Atmosphere Monitoring Service PM2.5, 2 day forecasts\n 24th December 2021 at 12:00 UTC", fontsize=18)
plt.savefig("CAMS-PM2_5-fc-20211224.png")

In [ ]:
list_times = np.datetime64('2021-12-22') + ds.time.sel(time=slice(np.timedelta64(0),np.timedelta64(1,'D')))
print(pd.to_datetime(list_times).strftime("%d %b %H:%S UTC"))

In [ ]:
fig = plt.figure(1, figsize=[10,10])

# We're using cartopy to project our data.
# (see documentation on cartopy)
proj_plot = ccrs.Mercator()

# We need to project our data to the new projection and for this we use `transform`.
# we set the original data projection in transform (here PlateCarree)
p = ds.sel(time=slice(np.timedelta64(1,'h'),np.timedelta64(1,'D')))['pm2p5_conc'].plot(transform=ccrs.PlateCarree(),
                                                                                       vmin = 0, vmax = 35,
                                                                                       subplot_kws={"projection": proj_plot},
                                                                                       col='time', col_wrap=4,
                                                                                       robust=True,
                                                                                      aspect=ds.dims["longitude"] / ds.dims["latitude"],  # for a sensible figsize
                                                                                       cmap=cmc.roma_r)
# We have to set the map's options on our axes
for ax,i in zip(p.axes.flat,  (np.datetime64('2021-12-22') + ds.time.sel(time=slice(np.timedelta64(0),np.timedelta64(1,'D')))).values):
      ax.coastlines('10m')
      ax.set_title("CAMS PM2.5 " + pd.to_datetime(i).strftime("%d %b %H:%S UTC"), fontsize=12)
# Save your figure
plt.savefig("CAMS-PM2_5-fc-multi.png")